In [30]:
from qiskit import IBMQ
# IBMQ.save_account(MY_API_TOKEN)
import qiskit 
qiskit.__version__

import numpy as np
import random
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator
from qiskit import(QuantumCircuit, execute, Aer)
from qiskit.visualization import plot_histogram
from qiskit.extensions import Initialize # Import the Inititialize function
from qiskit.aqua.circuits.gates import multi_control_toffoli_gate

In [131]:
visible = 2
hidden = 2

x = np.array([random.uniform(0, 1) for n in range(visible)])
theta = np.random.rand(visible, hidden) * np.pi
# [[np.pi/6, np.pi/4], [np.pi / 2, np.pi]]


In [134]:
qc = QuantumCircuit(visible + hidden, visible)
initial_state = [[np.sqrt(x[i]), np.sqrt(1-x[i])] for i in range(len(x))]
for i in range(visible):
    initialize_qubit = Initialize(initial_state[i])
    qc.append(initialize_qubit, [i])
    qc.h(i)
for i in range(hidden):
    for j in range(visible):
        qc.rz(theta[j][i], j)
    print(list(range(visible)) + [visible + i])
    multi_control_toffoli_gate._multicx(qc, list(range(visible)) + [visible + i])
for i in range(hidden):
    qc.h(visible + i)
# Map the quantum measurement to the classical bits
qc.measure(list(range(visible)), list(range(visible)))

[0, 1, 2]
[0, 1, 3]


In [136]:
# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator
job = execute(qc, simulator, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(qc)
print("\nTotal count for 0 and 1 are:",counts)
qc.draw()


Total count for 0 and 1 are: {'00': 796, '11': 8, '01': 69, '10': 127}


┌─────────────────────────────┐┌───┐┌─────────────┐     ┌────────────┐»
q_0: |0>┤ Initialize(0.87254,0.48854) ├┤ H ├┤ Rz(0.65225) ├──■──┤ Rz(0.3938) ├»
        ├─────────────────────────────┤├───┤└┬────────────┤  │  ├────────────┤»
q_1: |0>┤ Initialize(0.38749,0.92187) ├┤ H ├─┤ Rz(1.2429) ├──■──┤ Rz(1.5648) ├»
        └─────────────────────────────┘└───┘ └────────────┘┌─┴─┐└───┬───┬────┘»
q_2: |0>───────────────────────────────────────────────────┤ X ├────┤ H ├─────»
                                                           └───┘    └───┘     »
q_3: |0>──────────────────────────────────────────────────────────────────────»
                                                                              »
 c_0: 0 ══════════════════════════════════════════════════════════════════════»
                                                                              »
 c_1: 0 ══════════════════════════════════════════════════════════════════════»
                                                                              »
«               ┌─┐   
«q_0: ──■───────┤M├───
«       │       └╥┘┌─┐
«q_1: ──■────────╫─┤M├
«       │        ║ └╥┘
«q_2: ──┼────────╫──╫─
«     ┌─┴─┐┌───┐ ║  ║ 
«q_3: ┤ X ├┤ H ├─╫──╫─
«     └───┘└───┘ ║  ║ 
«c_0: ═══════════╩══╬═
«                   ║ 
«c_1: ══════════════╩═
«

In [137]:
visible = 2
hidden = 3
qc = QuantumCircuit(visible+hidden, visible)
for i in range(hidden):
#     for j in range(visible):
#         qc.rz(theta[j][i], j)
    print(list(range(visible)) + [visible + i])
    multi_control_toffoli_gate.mct(qc, list(range(visible)), visible + i, None)
    
qc.draw()

[0, 1, 2]
[0, 1, 3]
[0, 1, 4]


q_0: |0>──■────■────■──
          │    │    │  
q_1: |0>──■────■────■──
        ┌─┴─┐  │    │  
q_2: |0>┤ X ├──┼────┼──
        └───┘┌─┴─┐  │  
q_3: |0>─────┤ X ├──┼──
             └───┘┌─┴─┐
q_4: |0>──────────┤ X ├
                  └───┘
 c_0: 0 ═══════════════
                       
 c_1: 0 ═══════════════